### Build a logistic regression model and show the label& Predictions

In [89]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}

import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}


In [105]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline


In [91]:
val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [92]:
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [93]:
val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [94]:
val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [95]:
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [96]:
val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET")).filter(!_.contains("SK_ID_CURR")) //Make sure we exclude the ID column and label column

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [97]:
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   train_df=train_df.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [99]:
train_df.printSchema()

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_EMPLOYED: float (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: float (nullable = true)
 |-- bureau_DAYS_CREDIT_max: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_mean: float (nullable = true)
 |-- DAYS_REGISTRATION: 

 |-- previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_OVERDUE_mean: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Construction_count_norm: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_SCOFR_count_norm: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Furniture_count_norm: float (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max: float (nullable = true)
 |-- previous_loans_NAME_GOODS_CATEGORY_Construction Materials_count_norm: float (nullable = true)
 |-- client_bureau_balance_STATUS_1_count_norm_sum: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_min_min: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_VERIF_count_norm: float (nullable 

In [39]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val df2 = assembler.transform(train_df)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f08777cc4191
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 281 more fields]


In [40]:
df2.printSchema()

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_EMPLOYED: float (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: float (nullable = true)
 |-- bureau_DAYS_CREDIT_max: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_mean: float (nullable = true)
 |-- DAYS_REGISTRATION: 

 |-- previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_OVERDUE_mean: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Construction_count_norm: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_SCOFR_count_norm: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Furniture_count_norm: float (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max: float (nullable = true)
 |-- previous_loans_NAME_GOODS_CATEGORY_Construction Materials_count_norm: float (nullable = true)
 |-- client_bureau_balance_STATUS_1_count_norm_sum: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_min_min: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_VERIF_count_norm: float (nullable 

In [41]:
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_447cde7ed22b
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 282 more fields]


In [42]:
val model = new LogisticRegression().fit(df3)
val predictions = model.transform(df3)

2019-06-05 10:24:36,879 WARN  [Executor task launch worker for task 1017] memory.MemoryStore (Logging.scala:logWarning(66)) - Not enough space to cache rdd_745_0 in memory! (computed 27.3 MB so far)
2019-06-05 10:24:36,881 WARN  [Executor task launch worker for task 1017] storage.BlockManager (Logging.scala:logWarning(66)) - Persisting block rdd_745_0 to disk instead.


model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_7cdd3b22ff72, numClasses = 2, numFeatures = 280
predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 285 more fields]


In [62]:
predictions.select ("SK_ID_CURR","features", "label","probability", "prediction").show()

+----------+--------------------+-----+--------------------+----------+
|SK_ID_CURR|            features|label|         probability|prediction|
+----------+--------------------+-----+--------------------+----------+
|  162184.0|[1.11118999484460...|  0.0|[0.96453092256627...|       0.0|
|  123241.0|(280,[0,1,2,3,4,5...|  1.0|[0.92181159260730...|       0.0|
|  139370.0|(280,[0,1,2,3,4,5...|  1.0|[0.71883607996861...|       0.0|
|  148800.0|[5.80878986511379...|  0.0|[0.73065884568556...|       0.0|
|  130112.0|[6.54166971798986...|  1.0|[0.74178784254276...|       0.0|
|  103041.0|(280,[0,1,2,3,4,5...|  0.0|[0.84746555765860...|       0.0|
|  147356.0|(280,[0,1,2,3,4,5...|  0.0|[0.92333645707510...|       0.0|
|  135712.0|(280,[0,1,2,3,4,5...|  1.0|[0.45620231151929...|       1.0|
|  136748.0|(280,[0,1,2,3,4,5...|  0.0|[0.75632565380304...|       0.0|
|  151389.0|(280,[0,1,2,3,4,5...|  0.0|[0.85925346391027...|       0.0|
|  112328.0|(280,[0,1,2,3,4,5...|  0.0|[0.75219243655687...|    

### Instead of run each function with multiple transform on dataset, we can set the assembler, label Indexer and model into pipeline:

In [106]:
val lrPipeline = new Pipeline().setStages(Array(labelIndexer, assembler, model))

val lrModel = lrPipeline.fit(train_df)


lrPipeline: org.apache.spark.ml.Pipeline = pipeline_386d9fe8bdc2
lrModel: org.apache.spark.ml.PipelineModel = pipeline_386d9fe8bdc2


In [107]:
val predictions1 = lrModel.transform(train_df)
predictions1.select ("SK_ID_CURR","features", "label","probability", "prediction").show()

+----------+--------------------+-----+--------------------+----------+
|SK_ID_CURR|            features|label|         probability|prediction|
+----------+--------------------+-----+--------------------+----------+
|  145702.0|[1.73074993654154...|  0.0|[0.80239616107334...|       0.0|
|  139370.0|(280,[0,1,2,3,4,5...|  1.0|[0.71883607996861...|       0.0|
|  157586.0|[5.26803021784871...|  1.0|[0.83932866796578...|       0.0|
|  148800.0|[5.80878986511379...|  0.0|[0.73065884568556...|       0.0|
|  130112.0|[6.54166971798986...|  1.0|[0.74178784254276...|       0.0|
|  103041.0|(280,[0,1,2,3,4,5...|  0.0|[0.84746555765860...|       0.0|
|  135712.0|(280,[0,1,2,3,4,5...|  1.0|[0.45620231151929...|       1.0|
|  136748.0|(280,[0,1,2,3,4,5...|  0.0|[0.75632565380304...|       0.0|
|  151389.0|(280,[0,1,2,3,4,5...|  0.0|[0.85925346391027...|       0.0|
|  112328.0|(280,[0,1,2,3,4,5...|  0.0|[0.75219243655687...|       0.0|
|  125091.0|(280,[0,1,2,3,4,5...|  1.0|[0.61296886102941...|    

predictions1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 285 more fields]


### Note that 'prediction' and 'prediction1' are identicle, with the latter generated by pipeline
### Now we check the coefficients of each variable:

In [49]:
println(s"Coefficients: ${model.coefficients} Intercept: ${model.intercept}")

Coefficients: [-1.992623043807887,-2.193920990208578,-3.489685010059842E-7,3.424466598363812E-5,4.1708080577416654E-8,9.583023663582491E-6,-1.5153913535693228E-7,-1.986302613661628E-6,-5.298542530855307E-5,1.8967169212265913E-5,-6.832081959494841E-5,1.9746875671261832E-7,0.017828937523268678,0.01096661662569543,-4.319471054198385E-7,2.2214122060346098E-5,1.0520739565465583E-8,-1.425734650885446E-7,9.80198191892948E-6,-1.0991336577865482E-4,-9.521372233324889E-7,-0.29265703970391815,1.4012842789694349,0.003081905905888688,-2.5523355630706335E-4,1.2371225158052954E-8,-0.1264322437348097,1.6887200038933334E-5,-9.036243852669412E-7,-2.2252937564207274E-5,0.37463183567306296,-1.728708584912292E-7,0.030180735917681636,-0.018160148654549613,2.4336481726604288E-5,0.1381690129207532,-0.06915658426524468,5.631315118253187E-6,3.8242691669967325E-7,2.8831347593275535E-6,6.472362110549365E-5,-2.4810137030141314E-6,-3.607475948235999E-6,2.3176747612706724E-5,-6.962411509735002E-7,-0.6128359476220323

### Check the accuracy of the prediction:

In [103]:
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.SparkSession


In [45]:
    val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")
    val accuracy = evaluator.evaluate(predictions)
    println("Test set accuracy = " + accuracy)

Test set accuracy = 0.9277759631686187


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ba3a173d7867
accuracy: Double = 0.9277759631686187


### We can see that the accuracy of our prediction is over 90%. Much higher than our threshold 50%
### But the 'target' of our data is highly skewed, with most of them are 0. The accuracy might not be a good criteria for model testing

### Inspect further into the prediction with different label value(0 or 1):

In [81]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics


In [69]:
val lp = predictions.select( "label", "prediction")
val counttotal = predictions.count()
val correct = lp.filter($"label" === $"prediction").count() //correct prediction
val wrong = lp.filter(not($"label" === $"prediction")).count() //wrong prediction
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count() //Correct prediction of label=0
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when label=1
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when lable =0
val ratioWrong=wrong.toDouble/counttotal.toDouble //Wrong prediction ratio
val ratioCorrect=correct.toDouble/counttotal.toDouble //Correct predicition ratio

lp: org.apache.spark.sql.DataFrame = [label: double, prediction: double]
counttotal: Long = 128586
correct: Long = 119299
wrong: Long = 9287
truep: Long = 119098
falseN: Long = 9092
falseP: Long = 195
ratioWrong: Double = 0.07222403683138133
ratioCorrect: Double = 0.9277759631686187


### we use the binary classification metrics to evaluate the model

In [83]:
val predictionLabelsRDD = predictions.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1586] at map at <console>:114
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@72f3d318


In [86]:
println("area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

area under the receiver operating characteristic (ROC) curve : 0.5099972762785058
